# Neural Networks Assignment: Text Classification

**Student Name:** [Your Name Here]  
**Date:** [Date]

---

## Introduction

In this assignment, you'll apply everything you learned from MNIST to classify **movie reviews** as positive or negative.

**Key Insight:** This is the *same problem* as MNIST with different input!

| Aspect | MNIST | Text Classification |
|--------|-------|--------------------|
| **Input** | 28×28 image (784 pixels) | Review text (5,000 word features) |
| **Output** | 10 classes (digits 0-9) | 2 classes (positive/negative) |
| **Architecture** | Input → Hidden → Output | Input → Hidden → Output |
| **Training** | Mini-batch gradient descent | Mini-batch gradient descent |
| **Activation** | ReLU + Softmax | ReLU + Sigmoid |

**What's Different:** Instead of pixels, we use **word counts**. That's it!

---

## What You'll Do

✅ **Provided for you:** All data preprocessing, helper functions, visualization code  
🔧 **Your tasks:** Implement forward pass, loss function, complete training loop  
💭 **Reflection:** Understand what you built and how it works

**Estimated time:** 2-4 hours

Let's get started!

---
## Setup: Import Libraries

Run this cell first. If you get import errors, install missing packages.

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Set random seed for reproducibility
np.random.seed(42)

# Plotting style
plt.style.use('default')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print(f"NumPy version: {np.__version__}")

---
# Part 1: Understanding the Data

## 1.1 Sample Dataset (Provided)

We've created a simplified dataset of 500 short movie reviews for you.

In [ ]:
# Sample movie reviews (250 positive + 250 negative)
positive_reviews = [
    "This movie was absolutely fantastic and entertaining!",
    "Loved every minute of it, brilliant performance.",
    "An amazing film with great acting and plot.",
    "Wonderful story, beautifully shot, highly recommend.",
    "Excellent movie, one of the best I've seen.",
    "Incredible experience, the cast was perfect.",
    "Thoroughly enjoyed this film, very engaging.",
    "Superb direction and wonderful screenplay.",
    "Fantastic cinematography and compelling narrative.",
    "Outstanding performances, truly memorable film."
] * 25  # Repeat to get 250 samples

negative_reviews = [
    "Terrible movie, complete waste of time and money.",
    "Boring and predictable, couldn't finish watching.",
    "Awful acting and terrible plot, very disappointed.",
    "Worst film I've ever seen, avoid at all costs.",
    "Poorly written and badly executed, not recommended.",
    "Dreadful movie with no redeeming qualities.",
    "Horrible experience, regret watching this film.",
    "Pathetic storyline and unconvincing performances.",
    "Disappointingly bad, don't waste your time.",
    "Utterly boring and completely forgettable movie."
] * 25  # Repeat to get 250 samples

# Combine and create labels
all_reviews = positive_reviews + negative_reviews
labels = [1] * len(positive_reviews) + [0] * len(negative_reviews)  # 1 = positive, 0 = negative

print(f"📊 Dataset loaded: {len(all_reviews)} reviews")
print(f"   • Positive: {sum(labels)} reviews")
print(f"   • Negative: {len(labels) - sum(labels)} reviews")
print(f"\n✅ Dataset is balanced (50% positive, 50% negative)")

### Explore Sample Reviews

In [ ]:
# Show examples
print("📝 POSITIVE Review Examples:")
for i in range(3):
    print(f"   {i+1}. {positive_reviews[i]}")

print("\n📝 NEGATIVE Review Examples:")
for i in range(3):
    print(f"   {i+1}. {negative_reviews[i]}")

---
## 1.2 Text Preprocessing (All Provided!)

**This is the complex part - but we've done it for you!**

We need to convert text into numbers. Here's how:

1. **Clean text** (lowercase, remove punctuation)
2. **Tokenize** (split into words)
3. **Build vocabulary** (find most common words)
4. **Convert to Bag-of-Words** (count word occurrences)

Just run these cells and observe!

In [ ]:
# ============================================================================
# PROVIDED: Text Preprocessing Functions
# ============================================================================

def clean_text(text):
    """Clean and normalize text"""
    # Lowercase
    text = text.lower()
    # Remove punctuation and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

def tokenize(text):
    """Split text into words"""
    return text.split()

def build_vocabulary(texts, vocab_size=5000):
    """
    Build vocabulary of most common words.
    
    Returns:
        word_to_idx: Dictionary mapping words to indices
        idx_to_word: Dictionary mapping indices to words
    """
    # Count all words
    all_words = []
    for text in texts:
        all_words.extend(tokenize(clean_text(text)))
    
    # Get most common words
    word_counts = Counter(all_words)
    most_common = word_counts.most_common(vocab_size)
    
    # Create mappings
    word_to_idx = {word: idx for idx, (word, _) in enumerate(most_common)}
    idx_to_word = {idx: word for word, idx in word_to_idx.items()}
    
    return word_to_idx, idx_to_word

def text_to_bow(text, word_to_idx):
    """
    Convert text to Bag-of-Words vector.
    
    Example:
        "great movie" → [0, 0, 1, 0, 1, 0, ...] (5000 features)
                          position 2 = "great" appears 1 time
                          position 4 = "movie" appears 1 time
    """
    vector = np.zeros(len(word_to_idx))
    tokens = tokenize(clean_text(text))
    
    for word in tokens:
        if word in word_to_idx:
            vector[word_to_idx[word]] += 1
    
    return vector

print("✅ Preprocessing functions defined")

### Apply Preprocessing

In [ ]:
# Build vocabulary from all reviews
print("Building vocabulary...")
word_to_idx, idx_to_word = build_vocabulary(all_reviews, vocab_size=100)
vocab_size = len(word_to_idx)

print(f"\n✅ Vocabulary built: {vocab_size} unique words")
print(f"\nMost common words: {list(word_to_idx.keys())[:20]}")

### See How Text Becomes Numbers!

In [ ]:
# Example: Convert one review to numbers
example_review = positive_reviews[0]
example_vector = text_to_bow(example_review, word_to_idx)

print(f"📝 Original Review:")
print(f"   '{example_review}'")
print(f"\n🔢 After Bag-of-Words Conversion:")
print(f"   Vector shape: {example_vector.shape}")
print(f"   Non-zero elements: {np.count_nonzero(example_vector)}")
print(f"\n   First 20 values: {example_vector[:20]}")
print(f"\n💡 Interpretation: This review uses {int(np.count_nonzero(example_vector))} different words from our vocabulary.")

### Convert All Reviews to Numbers

In [ ]:
# Convert all reviews to Bag-of-Words vectors
print("Converting all reviews to numerical format...")
X = np.array([text_to_bow(review, word_to_idx) for review in all_reviews])
y = np.array(labels).reshape(-1, 1)  # Reshape to (500, 1)

print(f"\n✅ Conversion complete!")
print(f"   X shape: {X.shape}  (500 reviews × {vocab_size} word features)")
print(f"   y shape: {y.shape}  (500 labels × 1)")
print(f"\n💡 Compare to MNIST:")
print(f"   MNIST: X = (60000, 784)   → 60,000 images, 784 pixels each")
print(f"   Text:  X = (500, {vocab_size})      → 500 reviews, {vocab_size} word features each")

---
## 1.3 Train-Test Split

In [ ]:
# Split into training and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"📊 Data Split:")
print(f"   Training set: {X_train.shape[0]} reviews")
print(f"   Test set:     {X_test.shape[0]} reviews")
print(f"\n   X_train shape: {X_train.shape}")
print(f"   y_train shape: {y_train.shape}")
print(f"\n✅ Ready for training!")

---
# Part 2: Neural Network Implementation

## 2.1 Activation Functions (Provided)

In [ ]:
# ============================================================================
# PROVIDED: Activation Functions
# ============================================================================

def sigmoid(x):
    """Sigmoid activation: maps any value to (0, 1)"""
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))  # Clip to prevent overflow

def relu(x):
    """ReLU activation: max(0, x)"""
    return np.maximum(0, x)

def relu_derivative(x):
    """Derivative of ReLU"""
    return (x > 0).astype(float)

print("✅ Activation functions defined")

---
## 2.2 Neural Network Class

**This is where YOU come in!**

You'll implement 3 key methods:
1. `forward()` - How data flows through the network
2. `compute_loss()` - Measure prediction error
3. Complete the training loop (fill in missing lines)

The structure is provided - you fill in the TODOs!

In [ ]:
class TextClassifier:
    def __init__(self, input_size, hidden_size, output_size=1):
        """
        Initialize neural network for binary text classification.
        
        Architecture:
            Input (input_size) → Hidden (hidden_size, ReLU) → Output (1, Sigmoid)
        
        Args:
            input_size: Number of input features (vocabulary size)
            hidden_size: Number of neurons in hidden layer
            output_size: Number of output neurons (1 for binary classification)
        """
        # Initialize weights with small random values
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros((1, output_size))
        
        # Storage for training history
        self.train_loss_history = []
        self.test_loss_history = []
        self.train_acc_history = []
        self.test_acc_history = []
        
        print(f"🧠 Neural Network Initialized")
        print(f"   Architecture: {input_size} → {hidden_size} → {output_size}")
        print(f"   Total parameters: {input_size * hidden_size + hidden_size + hidden_size * output_size + output_size:,}")
    
    def forward(self, X):
        """
        Forward pass: Input → Hidden → Output
        
        TODO: Implement the forward pass!
        
        Steps:
            1. Compute hidden layer: z1 = X @ W1 + b1
            2. Apply ReLU activation: a1 = relu(z1)
            3. Compute output layer: z2 = a1 @ W2 + b2
            4. Apply sigmoid activation: a2 = sigmoid(z2)
        
        Hint: This is nearly IDENTICAL to MNIST forward pass!
        The only difference: sigmoid instead of softmax at the end.
        
        Args:
            X: Input data (batch_size, input_size)
        
        Returns:
            a2: Output predictions (batch_size, 1)
        """
        # TODO: Implement forward pass
        # Step 1: Hidden layer (before activation)
        self.z1 = None  # Replace None with your code
        
        # Step 2: Hidden layer (after ReLU activation)
        self.a1 = None  # Replace None with your code
        
        # Step 3: Output layer (before activation)
        self.z2 = None  # Replace None with your code
        
        # Step 4: Output layer (after sigmoid activation)
        self.a2 = None  # Replace None with your code
        
        return self.a2
    
    def compute_loss(self, y_true, y_pred):
        """
        Compute Binary Cross-Entropy Loss.
        
        TODO: Implement the loss function!
        
        Formula:
            loss = -1/m * Σ[y*log(ŷ) + (1-y)*log(1-ŷ)]
        
        Where:
            m = number of samples
            y = true labels (0 or 1)
            ŷ = predicted probabilities (0 to 1)
        
        Hint: Use np.log(), be careful of log(0)!
        Add small epsilon (1e-8) to avoid log(0).
        
        Args:
            y_true: True labels (m, 1)
            y_pred: Predicted probabilities (m, 1)
        
        Returns:
            loss: Single number (average loss)
        """
        m = y_true.shape[0]
        epsilon = 1e-8  # Small value to prevent log(0)
        
        # TODO: Implement binary cross-entropy loss
        # Clip predictions to avoid log(0)
        y_pred_clipped = np.clip(y_pred, epsilon, 1 - epsilon)
        
        # Calculate loss (replace the 0 with your formula)
        loss = 0  # Replace this with your implementation
        
        return loss
    
    def backward(self, X, y_true, learning_rate):
        """
        Backward pass: Compute gradients and update weights.
        
        PROVIDED: This is implemented for you!
        Study it - it's the same backpropagation from MNIST.
        """
        m = X.shape[0]
        
        # Output layer gradients
        dz2 = self.a2 - y_true  # For sigmoid + binary cross-entropy
        dW2 = np.dot(self.a1.T, dz2) / m
        db2 = np.sum(dz2, axis=0, keepdims=True) / m
        
        # Hidden layer gradients
        dz1 = np.dot(dz2, self.W2.T) * relu_derivative(self.z1)
        dW1 = np.dot(X.T, dz1) / m
        db1 = np.sum(dz1, axis=0, keepdims=True) / m
        
        # Update weights
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
    
    def predict(self, X):
        """
        Make predictions (0 or 1).
        
        PROVIDED: Converts probabilities to class labels.
        """
        probabilities = self.forward(X)
        return (probabilities >= 0.5).astype(int)

print("✅ TextClassifier class defined")
print("\n⚠️  Remember to implement the TODO sections in forward() and compute_loss()!")

---
## 2.3 Create the Network

In [ ]:
# Create network instance
model = TextClassifier(input_size=vocab_size, hidden_size=64, output_size=1)

print("\n📊 Network Architecture:")
print(f"   Input Layer:  {vocab_size} neurons (one per word in vocabulary)")
print(f"   Hidden Layer: 64 neurons (ReLU activation)")
print(f"   Output Layer: 1 neuron (Sigmoid activation → probability)")

---
# Part 3: Training

## 3.1 Training Loop

**You need to complete 3 lines in the training loop below!**

Look for `# TODO` comments.

In [ ]:
# Training hyperparameters
epochs = 100
learning_rate = 0.1
batch_size = 32

print(f"🏋️ Training Configuration:")
print(f"   Epochs: {epochs}")
print(f"   Learning Rate: {learning_rate}")
print(f"   Batch Size: {batch_size}")
print(f"\nStarting training...\n")

# Training loop
n_batches = len(X_train) // batch_size

for epoch in range(epochs):
    # Shuffle training data
    indices = np.random.permutation(len(X_train))
    X_shuffled = X_train[indices]
    y_shuffled = y_train[indices]
    
    # Mini-batch training
    for i in range(n_batches):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
        
        X_batch = X_shuffled[start_idx:end_idx]
        y_batch = y_shuffled[start_idx:end_idx]
        
        # TODO 1: Forward pass - make predictions on this batch
        # Hint: Use model.forward(X_batch)
        predictions = None  # Replace None with your code
        
        # TODO 2: Backward pass - update weights based on errors
        # Hint: Use model.backward(X_batch, y_batch, learning_rate)
        # YOUR CODE HERE
    
    # Evaluate every 10 epochs
    if (epoch + 1) % 10 == 0:
        # Predictions on full datasets
        train_pred = model.forward(X_train)
        test_pred = model.forward(X_test)
        
        # TODO 3: Calculate loss using model.compute_loss()
        # Hint: train_loss = model.compute_loss(y_train, train_pred)
        train_loss = None  # Replace None with your code
        test_loss = model.compute_loss(y_test, test_pred)
        
        # Calculate accuracy
        train_acc = np.mean((train_pred >= 0.5).astype(int) == y_train)
        test_acc = np.mean((test_pred >= 0.5).astype(int) == y_test)
        
        # Store history
        model.train_loss_history.append(train_loss)
        model.test_loss_history.append(test_loss)
        model.train_acc_history.append(train_acc)
        model.test_acc_history.append(test_acc)
        
        print(f"Epoch {epoch+1:3d}/{epochs} | "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
              f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

print("\n✅ Training complete!")

---
## 3.2 Visualize Training Progress

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss plot
axes[0].plot(range(10, epochs+1, 10), model.train_loss_history, 'b-', label='Training Loss', linewidth=2)
axes[0].plot(range(10, epochs+1, 10), model.test_loss_history, 'r-', label='Test Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Training vs Test Loss', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy plot
axes[1].plot(range(10, epochs+1, 10), [acc*100 for acc in model.train_acc_history], 'b-', label='Training Accuracy', linewidth=2)
axes[1].plot(range(10, epochs+1, 10), [acc*100 for acc in model.test_acc_history], 'r-', label='Test Accuracy', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy (%)', fontsize=12)
axes[1].set_title('Training vs Test Accuracy', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([0, 100])

plt.tight_layout()
plt.show()

print(f"\n📊 Final Results:")
print(f"   Training Accuracy: {model.train_acc_history[-1]*100:.2f}%")
print(f"   Test Accuracy:     {model.test_acc_history[-1]*100:.2f}%")

---
# Part 4: Evaluation

## 4.1 Detailed Metrics

In [ ]:
# Get predictions
y_pred = model.predict(X_test)

# Calculate metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("📊 Test Set Performance:")
print(f"\n   Accuracy:  {accuracy*100:.2f}%")
print(f"   Precision: {precision:.4f}")
print(f"   Recall:    {recall:.4f}")
print(f"   F1-Score:  {f1:.4f}")

print("\n💡 What these metrics mean:")
print(f"   • Accuracy:  {accuracy*100:.1f}% of predictions are correct")
print(f"   • Precision: When model predicts POSITIVE, it's right {precision*100:.1f}% of the time")
print(f"   • Recall:    Model finds {recall*100:.1f}% of all positive reviews")
print(f"   • F1-Score:  Harmonic mean of precision and recall")

## 4.2 Confusion Matrix

In [ ]:
# Create confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'])
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.show()

print("\n📊 Confusion Matrix Breakdown:")
print(f"   True Negatives:  {cm[0,0]} (correctly predicted negative)")
print(f"   False Positives: {cm[0,1]} (predicted positive, actually negative)")
print(f"   False Negatives: {cm[1,0]} (predicted negative, actually positive)")
print(f"   True Positives:  {cm[1,1]} (correctly predicted positive)")

## 4.3 Error Analysis

In [ ]:
# Find misclassified examples
test_reviews = [all_reviews[i] for i, label in enumerate(labels) if i < len(all_reviews)]
misclassified_indices = np.where(y_pred.flatten() != y_test.flatten())[0]

print(f"🔍 Error Analysis: {len(misclassified_indices)} misclassified reviews\n")

# Show first 5 errors
for i, idx in enumerate(misclassified_indices[:5]):
    true_label = "POSITIVE" if y_test[idx] == 1 else "NEGATIVE"
    pred_label = "POSITIVE" if y_pred[idx] == 1 else "NEGATIVE"
    prob = model.forward(X_test[idx:idx+1])[0, 0]
    
    print(f"Error #{i+1}:")
    print(f"   Review: [Sample from test set]")
    print(f"   True Label: {true_label}")
    print(f"   Predicted: {pred_label} (probability: {prob:.3f})")
    print()

---
# Part 5: Reflection Questions

## Question 1: Connection to MNIST (3 points)

**Answer the following in the markdown cell below:**

a) How is this text classification problem similar to MNIST digit classification?  
b) What are the key differences?  
c) What parts of the code stayed exactly the same?

---

**YOUR ANSWER HERE:**

(Write 3-5 sentences)

---

## Question 2: Understanding Bag-of-Words (4 points)

**Answer the following:**

a) Explain in your own words: What is the Bag-of-Words representation?  
b) Why do we limit the vocabulary to the most common words instead of using ALL words?  
c) What information do we LOSE when converting text to Bag-of-Words?  
d) How does the hidden layer help the network make better decisions?

---

**YOUR ANSWER HERE:**

(Write 4-6 sentences)

---

## Question 3: Model Performance (3 points)

**Analyze your results:**

a) What was your final test accuracy? Is it good? Why or why not?  
b) Did your model overfit? (Compare training vs test accuracy)  
c) Looking at the misclassified examples, why do you think the model made mistakes?

---

**YOUR ANSWER HERE:**

(Write 3-5 sentences)

---

## Question 4: Future Improvements (Optional - Bonus 3 points)

**Think like a data scientist:**

If you had more time, what would you try to improve the model? List 3 specific ideas and explain why each might help.

---

**YOUR IDEAS:**

1. ...
2. ...
3. ...

---

---
# Submission Checklist

Before submitting, make sure:

- [ ] All code cells run without errors (Run → Run All Cells)
- [ ] All TODO sections are implemented (forward, compute_loss, training loop)
- [ ] All plots are displayed
- [ ] All reflection questions are answered
- [ ] Final test accuracy is reported
- [ ] File is named correctly: `LastName_FirstName_TextClassification.ipynb`

**Good luck! 🚀**